# Airline delays 
## Bureau of Transportation Statistics
https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236   
https://www.bts.gov/topics/airlines-and-airports/understanding-reporting-causes-flight-delays-and-cancellations

2015 - 2019

### Additioinal sources
This might be useful in matching station codes to airports:
1. http://dss.ucar.edu/datasets/ds353.4/inventories/station-list.html
2. https://www.world-airport-codes.com/

In [3]:
# imports
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, NullType, ShortType, DateType, BooleanType, BinaryType
from pyspark.sql import SQLContext
from pyspark.sql import DataFrameNaFunctions
sqlContext = SQLContext(sc)


In [4]:
display(dbutils.fs.ls("dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data"))

path,name,size
dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data/2015.parquet/,2015.parquet/,0
dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data/2016.parquet/,2016.parquet/,0
dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data/2017.parquet/,2017.parquet/,0
dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data/2018.parquet/,2018.parquet/,0
dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data/2019.parquet/,2019.parquet/,0


In [5]:
airlines = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data/201*.parquet")
#display(airlines.sample(False, 0.00001))

In [6]:
  airlines.printSchema()

root
-- YEAR: integer (nullable = true)
-- QUARTER: integer (nullable = true)
-- MONTH: integer (nullable = true)
-- DAY_OF_MONTH: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- FL_DATE: date (nullable = true)
-- OP_UNIQUE_CARRIER: string (nullable = true)
-- ORIGIN_AIRPORT_ID: integer (nullable = true)
-- ORIGIN_AIRPORT_SEQ_ID: integer (nullable = true)
-- ORIGIN_CITY_MARKET_ID: integer (nullable = true)
-- ORIGIN: string (nullable = true)
-- ORIGIN_CITY_NAME: string (nullable = true)
-- ORIGIN_STATE_ABR: string (nullable = true)
-- ORIGIN_STATE_FIPS: integer (nullable = true)
-- ORIGIN_STATE_NM: string (nullable = true)
-- ORIGIN_WAC: integer (nullable = true)
-- DEST_AIRPORT_ID: integer (nullable = true)
-- DEST_AIRPORT_SEQ_ID: integer (nullable = true)
-- DEST_CITY_MARKET_ID: integer (nullable = true)
-- DEST: string (nullable = true)
-- DEST_CITY_NAME: string (nullable = true)
-- DEST_STATE_ABR: string (nullable = true)
-- DEST_STATE_FIPS: integer (nullable = true)
-- DEST_STATE_NM: string (nullable = true)
-- DEST_WAC: integer (nullable = true)
-- CRS_DEP_TIME: integer (nullable = true)
-- DEP_TIME: integer (nullable = true)
-- DEP_DELAY: double (nullable = true)
-- DEP_DELAY_NEW: double (nullable = true)
-- DEP_DEL15: double (nullable = true)
-- DEP_DELAY_GROUP: integer (nullable = true)
-- DEP_TIME_BLK: string (nullable = true)
-- TAXI_OUT: double (nullable = true)
-- WHEELS_OFF: integer (nullable = true)
-- WHEELS_ON: integer (nullable = true)
-- TAXI_IN: double (nullable = true)
-- CRS_ARR_TIME: integer (nullable = true)
-- ARR_TIME: integer (nullable = true)
-- ARR_DELAY: double (nullable = true)
-- ARR_DELAY_NEW: double (nullable = true)
-- ARR_DEL15: double (nullable = true)
-- ARR_DELAY_GROUP: integer (nullable = true)
-- ARR_TIME_BLK: string (nullable = true)
-- CANCELLED: boolean (nullable = true)
-- DIVERTED: boolean (nullable = true)
-- CRS_ELAPSED_TIME: double (nullable = true)
-- ACTUAL_ELAPSED_TIME: double (nullable = true)
-- AIR_TIME: double (nullable = true)
-- FLIGHTS: double (nullable = true)
-- DISTANCE: double (nullable = true)
-- DISTANCE_GROUP: integer (nullable = true)
-- CARRIER_DELAY: double (nullable = true)
-- WEATHER_DELAY: double (nullable = true)
-- NAS_DELAY: double (nullable = true)
-- SECURITY_DELAY: double (nullable = true)
-- LATE_AIRCRAFT_DELAY: double (nullable = true)

In [7]:
f'{airlines.count():,}'

Out[5]: '31,746,841'

In [8]:
#display(airlines.describe())

In [9]:
airlines.where('MONTH == "MONTH"').count()

Out[7]: 0

In [10]:
airlines.where(airlines.ARR_DELAY == null).count()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-702435227259429> in <module> 
 ----> 1 airlines . where ( airlines . ARR_DELAY == null ) . count ( ) 

 NameError : name 'null' is not defined

In [11]:
for year in range(2015, 2020):
  print(year , airlines.select('MONTH').where(f'YEAR == {year}').distinct().collect())

2015 [Row(MONTH=12), Row(MONTH=1), Row(MONTH=6), Row(MONTH=3), Row(MONTH=5), Row(MONTH=9), Row(MONTH=4), Row(MONTH=8), Row(MONTH=7), Row(MONTH=10), Row(MONTH=11), Row(MONTH=2)]
2016 [Row(MONTH=12), Row(MONTH=1), Row(MONTH=6), Row(MONTH=3), Row(MONTH=5), Row(MONTH=9), Row(MONTH=4), Row(MONTH=8), Row(MONTH=7), Row(MONTH=10), Row(MONTH=11), Row(MONTH=2)]
2017 [Row(MONTH=12), Row(MONTH=1), Row(MONTH=6), Row(MONTH=3), Row(MONTH=5), Row(MONTH=9), Row(MONTH=4), Row(MONTH=8), Row(MONTH=7), Row(MONTH=10), Row(MONTH=11), Row(MONTH=2)]
2018 [Row(MONTH=12), Row(MONTH=1), Row(MONTH=6), Row(MONTH=3), Row(MONTH=5), Row(MONTH=9), Row(MONTH=4), Row(MONTH=8), Row(MONTH=7), Row(MONTH=10), Row(MONTH=11), Row(MONTH=2)]
2019 [Row(MONTH=12), Row(MONTH=1), Row(MONTH=6), Row(MONTH=3), Row(MONTH=5), Row(MONTH=9), Row(MONTH=4), Row(MONTH=8), Row(MONTH=7), Row(MONTH=10), Row(MONTH=11), Row(MONTH=2)]

In [12]:
def nullDataFrame(df):
  null_feature_list = []
  count = df.count()
  for column in df.columns:
    nulls = df.filter(df[column].isNull()).count()
    nulls_perct = np.round((nulls/count)*100, 2)
    null_feature_list.append([column, nulls, nulls_perct])
  nullCounts_df = pd.DataFrame(np.array(null_feature_list), columns=['Feature_Name', 'Null_Counts', 'Percentage_Null_Counts'])
  return nullCounts_df

In [13]:
#airlines.orderBy("FL_DATE").show(5)


In [14]:
def is_Weekend(x):
  """
  Function to determine if a given day of the week is a weekend_day(Friday, Saturday, Sunday)
  """
  if   x < 5: 
    return 0
  else: 
    return 1

def is_RushHour(x):
  """
  Function to determine if a given time of the day is rush hour (1600-2100)
  """
  if (x != None) and (x >= 1600) and (x <= 2100): 
    return 1
  else: 
    return 0
 
def preprocessAirlines(df):
  cols_to_keep = ['MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER', 'ORIGIN', 'DEST', 'DEP_DELAY', 'DEP_TIME_BLK', 'ARR_DELAY', 'ARR_TIME_BLK', 'CRS_ELAPSED_TIME', 'DISTANCE',  'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'IS_WEEKEND', 'DEP_RUSH_HOUR', 'ARR_RUSH_HOUR']
  cols_to_remove = [x for x in df.columns if x not in cols_to_keep]
  df = df.orderBy("FL_DATE") 
  df = df.filter(df.CANCELLED == False)
  df = df.filter(df.DIVERTED == False)
  df = df.withColumn('CARRIER_DELAY', f.when(df.CARRIER_DELAY.isNotNull(), 1).otherwise(0))
  df = df.withColumn('WEATHER_DELAY', f.when(df.WEATHER_DELAY.isNotNull(), 1).otherwise(0))
  df = df.withColumn('NAS_DELAY', f.when(df.NAS_DELAY.isNotNull(), 1).otherwise(0))
  df = df.withColumn('SECURITY_DELAY', f.when(df.SECURITY_DELAY.isNotNull(), 1).otherwise(0))
  df = df.withColumn('LATE_AIRCRAFT_DELAY', f.when(df.LATE_AIRCRAFT_DELAY.isNotNull(), 1).otherwise(0))
  df = df.withColumn("IS_WEEKEND", f.udf(is_Weekend, IntegerType())("DAY_OF_WEEK"))
  df = df.withColumn("DEP_RUSH_HOUR", f.udf(is_RushHour, IntegerType())("DEP_TIME"))
  df = df.withColumn("ARR_RUSH_HOUR", f.udf(is_RushHour, IntegerType())("CRS_ARR_TIME"))
  preprocessAirlines_df = df.drop(*cols_to_remove)
  return preprocessAirlines_df

In [15]:
airlines_df =  preprocessAirlines(airlines)
airlines_df.printSchema()

root
-- MONTH: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- OP_UNIQUE_CARRIER: string (nullable = true)
-- ORIGIN: string (nullable = true)
-- DEST: string (nullable = true)
-- DEP_DELAY: double (nullable = true)
-- DEP_TIME_BLK: string (nullable = true)
-- ARR_DELAY: double (nullable = true)
-- ARR_TIME_BLK: string (nullable = true)
-- CRS_ELAPSED_TIME: double (nullable = true)
-- DISTANCE: double (nullable = true)
-- CARRIER_DELAY: integer (nullable = false)
-- WEATHER_DELAY: integer (nullable = false)
-- NAS_DELAY: integer (nullable = false)
-- SECURITY_DELAY: integer (nullable = false)
-- LATE_AIRCRAFT_DELAY: integer (nullable = false)
-- IS_WEEKEND: integer (nullable = true)
-- DEP_RUSH_HOUR: integer (nullable = true)
-- ARR_RUSH_HOUR: integer (nullable = true)

In [16]:
display(airlines_df.sample(False, 0.0000001))

MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,ORIGIN,DEST,DEP_DELAY,DEP_TIME_BLK,ARR_DELAY,ARR_TIME_BLK,CRS_ELAPSED_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,IS_WEEKEND,DEP_RUSH_HOUR,ARR_RUSH_HOUR
9,4,B6,CHS,DCA,-15.0,1500-1559,-12.0,1600-1659,92.0,444.0,0,0,0,0,0,0,0,1
6,7,B6,JFK,PBI,-8.0,1300-1359,-37.0,1700-1759,187.0,1028.0,0,0,0,0,0,1,0,1
7,1,AA,RSW,PHL,-5.0,0600-0659,-13.0,0900-0959,159.0,993.0,0,0,0,0,0,0,0,0


In [17]:
type(airlines_df)

Out[200]: pyspark.sql.dataframe.DataFrame

In [18]:
nullCounts_df = nullDataFrame(airlines_df)
nullCounts_df

,Feature_Name,Null_Counts,Percentage_Null_Counts
0,MONTH,0,0.0
1,DAY_OF_WEEK,0,0.0
2,OP_UNIQUE_CARRIER,0,0.0
3,ORIGIN,0,0.0
4,DEST,0,0.0
5,DEP_DELAY,4725,0.02
6,DEP_TIME_BLK,0,0.0
7,ARR_DELAY,2600,0.01
8,ARR_TIME_BLK,0,0.0
9,CRS_ELAPSED_TIME,0,0.0


In [19]:
# 81.73% of records have null/NaN values for [CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY ]
airlines_preprocessed_filtered = airlines_df.filter(airlines['ARR_DELAY'].isNotNull() )
airlines_df.filter(airlines['ARR_DELAY'].isNull() ).show()

airlines_preprocessed_filtered = airlines_preprocessed_filtered.filter(airlines['DEP_DELAY'].isNotNull() )
f'{airlines_preprocessed_filtered.count():,}'

+-----+-----------+-----------------+------+----+---------+------------+---------+------------+----------------+--------+-------------+-------------+---------+--------------+-------------------+----------+-------------+-------------+
MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|ORIGIN|DEST|DEP_DELAY|DEP_TIME_BLK|ARR_DELAY|ARR_TIME_BLK|CRS_ELAPSED_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|IS_WEEKEND|DEP_RUSH_HOUR|ARR_RUSH_HOUR|
+-----+-----------+-----------------+------+----+---------+------------+---------+------------+----------------+--------+-------------+-------------+---------+--------------+-------------------+----------+-------------+-------------+
 1| 4| AA| TUL| DFW| null| 0600-0659| null| 0800-0859| 75.0| 237.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0| 0| 0|
 1| 4| AA| BWI| DFW| null| 1600-1659| null| 1800-1859| 215.0| 1217.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0| 0| 1|
 1| 4| AA| DFW| BWI| null| 1100-1159| null| 1500-1559| 165.0| 1217.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0| 0| 0|
 1| 4| AA| DFW| MFE| null| 0900-0959| null| 1000-1059| 95.0| 469.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0| 0| 0|
 1| 4| AA| DFW| SNA| null| 0700-0759| null| 0900-0959| 200.0| 1205.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0| 0| 0|
 1| 4| AA| LGA| DFW| null| 0600-0659| null| 0900-0959| 245.0| 1389.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0| 0| 0|
 1| 4| AA| DFW| IAH| null| 1000-1059| null| 1100-1159| 65.0| 224.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0| 0| 0|
 1| 4| AA| DFW| AUS| null| 1100-1159| null| 1200-1259| 60.0| 190.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0| 0| 0|
 1| 4| AA| DTW| DFW| null| 0600-0659| null| 0800-0859| 185.0| 986.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0| 0| 0|
 1| 4| AA| DFW| PHX| null| 0900-0959| null| 1100-1159| 160.0| 868.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0| 0| 0|
 1| 4| AA| DFW| MSP| null| 1000-1059| null| 1300-1359| 140.0| 852.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0| 0| 0|
 1| 4| AA| ATL| DFW| null| 0600-0659| null| 0700-0759| 150.0| 731.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0| 0| 0|
 1| 4| AA| DFW| DCA| null| 0600-0659| null| 0900-0959| 164.0| 1192.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0| 0| 0|
 1| 4| AA| ABQ| DFW| null| 1200-1259| null| 1400-1459| 100.0| 569.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0| 0| 0|
 1| 4| AA| DFW| ABQ| null| 1000-1059| null| 1100-1159| 110.0| 569.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0| 0| 0|
 1| 4| AA| MEM| DFW| null| 1000-1059| null| 1100-1159| 100.0| 432.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0| 0| 0|
 1| 4| AA| AUS| DFW| null| 0700-0759| null| 0800-0859| 70.0| 190.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0| 0| 0|
 1| 4| AA| DFW| FLL| null| 1000-1059| null| 1400-1459| 160.0| 1119.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0| 0| 0|
 1| 4| AA| DFW| LAS| null| 0900-0959| null| 1000-1059| 175.0| 1055.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0| 0| 0|
 1| 4| AA| TUS| DFW| null| 0700-0759| null| 1000-1059| 130.0| 813.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0| 0| 0|
+-----+-----------+-----------------+------+----+---------+------------+---------+------------+----------------+--------+-------------+-------------+---------+--------------+-------------------+----------+-------------+-------------+
only showing top 20 rows

Out[158]: '31,171,613'

In [20]:
display(airlines.filter(airlines['ARR_DELAY'].isNull() ))

YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
2019,3,7,20,6,2019-07-20,NK,14747,1474703,30559,SEA,"Seattle, WA",WA,53,Washington,93,12266,1226603,31453,IAH,"Houston, TX",TX,48,Texas,74,55,null,null,null,null,null,0001-0559,null,null,null,null,704,null,null,null,null,null,0700-0759,true,false,249.0,null,null,1.0,1874.0,8,null,null,null,null,null
2019,3,7,7,7,2019-07-07,NK,10158,1015804,30158,ACY,"Atlantic City, NJ",NJ,34,New Jersey,21,13577,1357702,31135,MYR,"Myrtle Beach, SC",SC,45,South Carolina,37,725,null,null,null,null,null,0700-0759,null,null,null,null,850,null,null,null,null,null,0800-0859,true,false,85.0,null,null,1.0,466.0,2,null,null,null,null,null
2019,3,7,26,5,2019-07-26,NK,11697,1169706,32467,FLL,"Fort Lauderdale, FL",FL,12,Florida,33,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,2059,null,null,null,null,null,2000-2059,null,null,null,null,2359,null,null,null,null,null,2300-2359,true,false,180.0,null,null,1.0,1127.0,5,null,null,null,null,null
2019,3,7,9,2,2019-07-09,NK,15304,1530402,33195,TPA,"Tampa, FL",FL,12,Florida,33,10821,1082106,30852,BWI,"Baltimore, MD",MD,24,Maryland,35,1130,1127,-3.0,0.0,0.0,-1,1100-1159,103.0,1310,1706,7.0,1349,1713,null,null,null,null,1300-1359,false,true,139.0,null,null,1.0,842.0,4,null,null,null,null,null
2019,3,7,30,2,2019-07-30,NK,10874,1087402,30647,CAK,"Akron, OH",OH,39,Ohio,44,13204,1320402,31454,MCO,"Orlando, FL",FL,12,Florida,33,1340,1325,-15.0,0.0,0.0,-1,1300-1359,7.0,1332,1758,12.0,1605,1810,null,null,null,null,1600-1659,false,true,145.0,null,null,1.0,861.0,4,null,null,null,null,null
2019,3,7,12,5,2019-07-12,NK,12889,1288903,32211,LAS,"Las Vegas, NV",NV,32,Nevada,85,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,1110,null,null,null,null,null,1100-1159,null,null,null,null,1811,null,null,null,null,null,1800-1859,true,false,241.0,null,null,1.0,1749.0,7,null,null,null,null,null
2019,3,7,23,2,2019-07-23,NK,10821,1082106,30852,BWI,"Baltimore, MD",MD,24,Maryland,35,13204,1320402,31454,MCO,"Orlando, FL",FL,12,Florida,33,610,null,null,null,null,null,0600-0659,null,null,null,null,827,null,null,null,null,null,0800-0859,true,false,137.0,null,null,1.0,787.0,4,null,null,null,null,null
2019,3,7,23,2,2019-07-23,NK,13204,1320402,31454,MCO,"Orlando, FL",FL,12,Florida,33,10821,1082106,30852,BWI,"Baltimore, MD",MD,24,Maryland,35,922,null,null,null,null,null,0900-0959,null,null,null,null,1139,null,null,null,null,null,1100-1159,true,false,137.0,null,null,1.0,787.0,4,null,null,null,null,null
2019,3,7,1,1,2019-07-01,NK,12892,1289208,32575,LAX,"Los Angeles, CA",CA,6,California,91,10397,1039707,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,1044,null,null,null,null,null,1000-1059,null,null,null,null,1825,null,null,null,null,null,1800-1859,true,false,281.0,null,null,1.0,1947.0,8,null,null,null,null,null
2019,3,7,8,1,2019-07-08,NK,12898,1289806,32898,LBE,"Latrobe, PA",PA,42,Pennsylvania,23,13577,1357702,31135,MYR,"Myrtle Beach, SC",SC,45,South Carolina,37,1540,null,null,null,null,null,1500-1559,null,null,null,null,1706,null,null,null,null,null,1700-1759,true,false,86.0,null,null,1.0,455.0,2,null,null,null,null,null


In [21]:
nullCounts_df2 = nullDataFrame(airlines_preprocessed_filtered)
nullCounts_df2

,Feature_Name,Null_Counts,Percentage_Null_Counts
0,MONTH,0,0.0
1,DAY_OF_WEEK,0,0.0
2,OP_UNIQUE_CARRIER,0,0.0
3,ORIGIN,0,0.0
4,DEST,0,0.0
5,DEP_DELAY,0,0.0
6,DEP_TIME_BLK,0,0.0
7,ARR_DELAY,0,0.0
8,ARR_TIME_BLK,0,0.0
9,CRS_ELAPSED_TIME,0,0.0


In [22]:
display(airlines_preprocessed_filtered.describe())

summary,MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,ORIGIN,DEST,DEP_DELAY,DEP_TIME_BLK,ARR_DELAY,ARR_TIME_BLK,CRS_ELAPSED_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,IS_WEEKEND,DEP_RUSH_HOUR,ARR_RUSH_HOUR
count,31176201,31176201,31176201,31176201,31176201,31171613,31176201,31176201,31176201,31176201,31176201,5799114,5799114,5799114,5799114,5799114,31176201,31176201,31176201
mean,6.568153669525033,3.936863506878211,null,null,null,9.784760320231102,null,4.615475952313754,null,143.39964208596166,825.1078288210933,19.98459350859459,3.2259498606166392,15.44036813209742,0.08916793151505557,25.364284785572416,0.4127280293067138,0.2794518164673111,0.3078280448602445
stddev,3.39745307682337,1.9914611932350432,null,null,null,43.31986119785449,null,45.59418015238945,null,74.8523322855301,608.6626085549944,59.307979706257626,26.812025382335808,34.739082338772576,2.9147981743398224,48.60358147038255,0.4923247006863924,0.4487298799936953,0.4615950026724097
min,1,1,9E,ABE,ABE,-234.0,0001-0559,-238.0,0001-0559,-99.0,28.0,0.0,0.0,0.0,0.0,0.0,0,0,0
max,12,7,YX,YUM,YUM,2710.0,2300-2359,2695.0,2300-2359,813.0,5095.0,2695.0,2692.0,1848.0,1078.0,2454.0,1,1,1


In [23]:
(airlines.select('ORIGIN').distinct().count())


Out[173]: 371

In [24]:
(airlines.select('DEST').distinct().count())

Out[172]: 369

In [25]:
(airlines.select('OP_UNIQUE_CARRIER').distinct().count())

Out[174]: 19

In [26]:
numeric_features = [x[0] for x in airlines_preprocessed_filtered.dtypes if x[1] == 'int' or x[1] == 'double']
numeric_features.remove('ARR_DELAY')
numeric_features

Out[88]: ['MONTH',
 'DAY_OF_WEEK',
 'DEP_DELAY',
 'CRS_ELAPSED_TIME',
 'DISTANCE',
 'CARRIER_DELAY',
 'WEATHER_DELAY',
 'NAS_DELAY',
 'SECURITY_DELAY',
 'LATE_AIRCRAFT_DELAY',
 'IS_WEEKEND',
 'DEP_RUSH_HOUR',
 'ARR_RUSH_HOUR']

In [27]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = numeric_features, outputCol = 'features')
vector_airlines_preprocessed_filtered = vectorAssembler.transform(airlines_preprocessed_filtered)
vector_airlines_preprocessed_filtered = vector_airlines_preprocessed_filtered.select(['features', 'ARR_DELAY'])
#display(vector_airlines_preprocessed_filtered.show())
display(vector_airlines_preprocessed_filtered)

features,ARR_DELAY
"List(0, 13, List(0, 1, 2, 3, 4, 11), List(1.0, 4.0, -6.0, 160.0, 733.0, 1.0))",-22.0
"List(0, 13, List(0, 1, 2, 3, 4, 12), List(1.0, 4.0, -4.0, 120.0, 733.0, 1.0))",-11.0
"List(0, 13, List(0, 1, 2, 3, 4, 5, 7), List(1.0, 4.0, 24.0, 210.0, 1192.0, 24.0, 13.0))",37.0
"List(0, 13, List(0, 1, 3, 4, 12), List(1.0, 4.0, 120.0, 733.0, 1.0))",1.0
"List(0, 13, List(0, 1, 2, 3, 4, 12), List(1.0, 4.0, -3.0, 120.0, 733.0, 1.0))",-11.0
"List(0, 13, List(0, 1, 2, 3, 4, 5, 7), List(1.0, 4.0, 12.0, 185.0, 1171.0, 12.0, 8.0))",20.0
"List(0, 13, List(0, 1, 2, 3, 4, 12), List(1.0, 4.0, 8.0, 175.0, 1171.0, 1.0))",-1.0
"List(0, 13, List(0, 1, 2, 3, 4), List(1.0, 4.0, -8.0, 338.0, 2724.0))",1.0
"List(1, 13, List(), List(1.0, 4.0, 78.0, 65.0, 247.0, 12.0, 0.0, 4.0, 0.0, 66.0, 0.0, 1.0, 1.0))",82.0
"List(1, 13, List(), List(1.0, 4.0, 70.0, 70.0, 247.0, 0.0, 0.0, 15.0, 0.0, 70.0, 0.0, 1.0, 1.0))",85.0


In [28]:
train_df, val_df, test_df = vector_airlines_preprocessed_filtered.randomSplit([0.8,0.1,0.1], seed = 2020)
display(train_df.show(5))

+--------------------+---------+
 features|ARR_DELAY|
+--------------------+---------+
(13,[0,1,2,3,4],[...| -24.0|
(13,[0,1,2,3,4],[...| -11.0|
(13,[0,1,2,3,4],[...| -50.0|
(13,[0,1,2,3,4],[...| -22.0|
(13,[0,1,2,3,4],[...| 11.0|
+--------------------+---------+
only showing top 5 rows

In [29]:
# train_df, val_df, test_df = airlines_preprocessed_filtered.randomSplit([0.8,0.1,0.1], seed = 2020)

In [30]:
train_cnt = train_df.count()
val_cnt = val_df.count()
test_cnt = test_df.count()
total_cnt = train_cnt + val_cnt + test_cnt

In [31]:
print('train_df records: {}\n val_df records: {}\n test_df records: {}\n total records: {}'.format(train_cnt, val_cnt, test_cnt, total_cnt) )

train_df records: 24938426
 val_df records: 3118360
 test_df records: 3115112
 total records: 31171898

In [32]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='ARR_DELAY', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.0,0.0,0.4211903047812193,-0.018296521211502433,0.0,0.5898021764672349,0.5883259293449599,0.7281103644718778,0.43989252234118625,0.5885881754107724,0.0,0.0,0.0]
Intercept: -4.302794128595644

In [33]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 9.962166
r2: 0.952292

In [34]:
print("numIterations: %d" % trainingSummary.totalIterations)
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

numIterations: 11
RMSE: 9.962166
r2: 0.952292

In [35]:
from  pyspark.sql.functions import abs
residuals = trainingSummary.residuals
abs_residuals = residuals.withColumn('abs_residuals',abs(residuals.residuals))
display(abs_residuals.show(5))
display(abs_residuals.describe().show())

+-------------------+------------------+
 residuals| abs_residuals|
+-------------------+------------------+
 -7.447575257992643| 7.447575257992643|
 4.52559765266324| 4.52559765266324|
 -31.80348056737407| 31.80348056737407|
-10.116150702259072|10.116150702259072|
 23.743785794681543|23.743785794681543|
+-------------------+------------------+
only showing top 5 rows

+-------+--------------------+--------------------+
summary| residuals| abs_residuals|
+-------+--------------------+--------------------+
 count| 24938609| 24938609|
 mean|0.001438111451416...| 7.784074008904573|
 stddev| 9.961383968131834| 6.2158959147476|
 min| -259.19017302989596|3.470878709777025...|
 max| 179.6729745039496| 259.19017302989596|
+-------+--------------------+--------------------+

In [36]:
abs_residuals_summary = abs_residuals.describe()
display(abs_residuals_summary)

summary,residuals,abs_residuals
count,24938973,24938973
mean,5.647702874134274E-4,7.784774118956812
stddev,9.96341009774851,6.218265882194345
min,-255.19745577605954,3.4708787097770255E-6
max,197.40831011880897,255.19745577605954


In [37]:
MEA_train_df = abs_residuals_summary.select('abs_residuals').toPandas().iloc[1]
print('Mean Absolute Error of train_df: ', MEA_train_df)

Mean Absolute Error of train_df: abs_residuals 7.784774118956812
Name: 1, dtype: object

In [38]:
lr_predictions_val = lr_model.transform(val_df)
lr_predictions_val.select("prediction","ARR_DELAY","features").show(5)

from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="ARR_DELAY",metricName="r2")
print("R Squared (R2) on val_data = %g" % lr_evaluator.evaluate(lr_predictions_val))

+-------------------+---------+--------------------+
 prediction|ARR_DELAY| features|
+-------------------+---------+--------------------+
 -12.83563871765572| -23.0|(13,[0,1,2,3,4],[...|
-14.975591065568175| -4.0|(13,[0,1,2,3,4],[...|
 -14.07832089237123| -15.0|(13,[0,1,2,3,4],[...|
-11.864071508862772| -1.0|(13,[0,1,2,3,4],[...|
-12.614228878534373| -22.0|(13,[0,1,2,3,4],[...|
+-------------------+---------+--------------------+
only showing top 5 rows

R Squared (R2) on val_data = 0.952302

In [39]:
lr_predictions_val.show(5)

+--------------------+---------+-------------------+
 features|ARR_DELAY| prediction|
+--------------------+---------+-------------------+
(13,[0,1,2,3,4],[...| -23.0| -12.83563871765572|
(13,[0,1,2,3,4],[...| -4.0|-14.975591065568175|
(13,[0,1,2,3,4],[...| -15.0| -14.07832089237123|
(13,[0,1,2,3,4],[...| -1.0|-11.864071508862772|
(13,[0,1,2,3,4],[...| -22.0|-12.614228878534373|
+--------------------+---------+-------------------+
only showing top 5 rows

In [40]:
lr_predictions_test = lr_model.transform(test_df)
lr_predictions_test.select("prediction","ARR_DELAY","features").show(5)

from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="ARR_DELAY",metricName="r2")
print("R Squared (R2) on val_data = %g" % lr_evaluator.evaluate(lr_predictions_test))

+-------------------+---------+--------------------+
 prediction|ARR_DELAY| features|
+-------------------+---------+--------------------+
-14.410250492594919| -16.0|(13,[0,1,2,3,4],[...|
-14.353879661293757| -19.0|(13,[0,1,2,3,4],[...|
-13.310607635321453| -9.0|(13,[0,1,2,3,4],[...|
-11.827848783356432| -11.0|(13,[0,1,2,3,4],[...|
-10.656501108903612| -24.0|(13,[0,1,2,3,4],[...|
+-------------------+---------+--------------------+
only showing top 5 rows

R Squared (R2) on val_data = 0.952011

In [41]:
sample_airlines_df = airlines_preprocessed_filtered.sample(False, 0.0001, 2020)
pandas_airlines_df = sample_airlines_df.toPandas()

In [42]:
#pandas_airlines_df[['ARR_DELAY', 'ARR_TIME', 'DEP_DELAY', 'DEP_TIME', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']][pandas_airlines_df.DEP_DELAY < 0 ].head(20)

In [43]:
nullCounts_df2 = nullDataFrame(airlines_preprocessed_filtered)
nullCounts_df2

,Feature_Name,Null_Counts,Percentage_Null_Counts
0,MONTH,0,0.0
1,DAY_OF_WEEK,0,0.0
2,OP_UNIQUE_CARRIER,0,0.0
3,ORIGIN,0,0.0
4,DEST,0,0.0
5,DEP_DELAY,4588,0.01
6,DEP_TIME_BLK,0,0.0
7,ARR_DELAY,0,0.0
8,ARR_TIME_BLK,0,0.0
9,CRS_ELAPSED_TIME,0,0.0


In [44]:
features = ['ARR_DELAY', 'DEP_DELAY', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'IS_WEEKEND', 'DEP_RUSH_HOUR', 'ARR_RUSH_HOUR']
# airlines_filtered[Delay_List].describe().show()

In [45]:
sample_airlines_df = airlines_preprocessed_filtered.select(features).sample(False, 0.001, 2020)
pandas_df = sample_airlines_df.toPandas()

In [46]:
pandas_df.corr()

,ARR_DELAY,DEP_DELAY,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,IS_WEEKEND,DEP_RUSH_HOUR,ARR_RUSH_HOUR
ARR_DELAY,1.000000,0.953464,0.640308,0.248526,0.288776,0.009273,0.491252,-0.006583,0.060950,0.053482
DEP_DELAY,0.953464,1.000000,0.645558,0.230005,0.177649,0.015039,0.521432,-0.000017,0.061483,0.049762
CARRIER_DELAY,0.640308,0.645558,1.000000,-0.040552,-0.096505,-0.012682,-0.085862,0.035752,-0.053316,-0.056121
WEATHER_DELAY,0.248526,0.230005,-0.040552,1.000000,-0.007366,-0.004658,-0.034703,-0.001323,0.001380,-0.008848
NAS_DELAY,0.288776,0.177649,-0.096505,-0.007366,1.000000,-0.014785,-0.145014,-0.043513,0.003826,0.054213
SECURITY_DELAY,0.009273,0.015039,-0.012682,-0.004658,-0.014785,1.000000,-0.019517,0.012748,-0.024240,-0.016260
LATE_AIRCRAFT_DELAY,0.491252,0.521432,-0.085862,-0.034703,-0.145014,-0.019517,1.000000,-0.010223,0.053879,0.035619
IS_WEEKEND,-0.006583,-0.000017,0.035752,-0.001323,-0.043513,0.012748,-0.010223,1.000000,0.001464,0.002057
DEP_RUSH_HOUR,0.060950,0.061483,-0.053316,0.001380,0.003826,-0.024240,0.053879,0.001464,1.000000,0.422751
ARR_RUSH_HOUR,0.053482,0.049762,-0.056121,-0.008848,0.054213,-0.016260,0.035619,0.002057,0.422751,1.000000


In [47]:
fig = plt.figure(figsize = (15,20))
ax = fig.gca()
pandas_df.hist(ax=ax, bins=30, figsize=(3,15),)
#ax.set_yscale('log')
plt.yscale('log')
display(plt.show())

# Weather
https://data.nodc.noaa.gov/cgi-bin/iso?id=gov.noaa.ncdc:C00532

In [49]:
dbutils.fs.ls("dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_weather_data")

Out[21]: [FileInfo(path='dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_weather_data/2015a.parquet/', name='2015a.parquet/', size=0),
 FileInfo(path='dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_weather_data/2016a.parquet/', name='2016a.parquet/', size=0),
 FileInfo(path='dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_weather_data/2017a.parquet/', name='2017a.parquet/', size=0),
 FileInfo(path='dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_weather_data/2018a.parquet/', name='2018a.parquet/', size=0),
 FileInfo(path='dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_weather_data/2019a.parquet/', name='2019a.parquet/', size=0)]

In [50]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, NullType
schema = StructType([StructField('STATION', StringType(), True), 
                      StructField('DATE', StringType(), True),
                      StructField('SOURCE', StringType(), True),
                      StructField('LATITUDE', StringType(), True),
                      StructField('LONGITUDE', StringType(), True),
                      StructField('ELEVATION', StringType(), True),
                      StructField('NAME', StringType(), True),
                      StructField('REPORT_TYPE', StringType(), True),
                      StructField('CALL_SIGN', StringType(), True),
                      StructField('QUALITY_CONTROL', StringType(), True),
                      StructField('WND', StringType(), True),
                      StructField('CIG', StringType(), True),
                      StructField('VIS', StringType(), True),
                      StructField('TMP', StringType(), True),
                      StructField('DEW', StringType(), True),
                      StructField('SLP', StringType(), True),
                      StructField('AA1', StringType(), True),
                      StructField('AA2', StringType(), True),
                      StructField('AJ1', StringType(), True),
                      StructField('AY1', StringType(), True),
                      StructField('AY2', StringType(), True),
                      StructField('GA1', StringType(), True),
                      StructField('GA2', StringType(), True),
                      StructField('GA3', StringType(), True),
                      StructField('GE1', StringType(), True),
                      StructField('GF1', StringType(), True),
                      StructField('IA1', StringType(), True),
                      StructField('KA1', StringType(), True),
                      StructField('KA2', StringType(), True),
                      StructField('MA1', StringType(), True),
                      StructField('MD1', StringType(), True),
                      StructField('MW1', StringType(), True),
                      StructField('OC1', StringType(), True),
                      StructField('OD1', StringType(), True),
                      StructField('SA1', StringType(), True),
                      StructField('UA1', StringType(), True),
                      StructField('REM', StringType(), True),
                      StructField('EQD', StringType(), True)
                    ])



In [51]:
weather = spark.read.option("header", "true")\
                      .schema(schema)\
                      .parquet(f"dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_weather_data/201*a.parquet")
weather.count()


Out[23]: 626994336

In [52]:
display(weather.where('DATE =="DATE"'))

STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,WND,CIG,VIS,TMP,DEW,SLP,AA1,AA2,AJ1,AY1,AY2,GA1,GA2,GA3,GE1,GF1,IA1,KA1,KA2,MA1,MD1,MW1,OC1,OD1,SA1,UA1,REM,EQD


In [53]:
#display(weather.describe())

In [54]:
nullCounts_weather_df = nullDataFrame(weather)
nullCounts_weather_df

,Feature_Name,Null_Counts,Percentage_Null_Counts
0,STATION,0,0.0
1,DATE,0,0.0
2,SOURCE,0,0.0
3,LATITUDE,0,0.0
4,LONGITUDE,0,0.0
5,ELEVATION,0,0.0
6,NAME,4715523,0.75
7,REPORT_TYPE,0,0.0
8,CALL_SIGN,0,0.0
9,QUALITY_CONTROL,0,0.0


In [55]:
#display(weather.sample(False, 0.0000001))

In [56]:
weather[["DATE"]].describe().show()

+-------+-------------------+
summary| DATE|
+-------+-------------------+
 count| 626994336|
 mean| null|
 stddev| null|
 min|2015-01-01T00:00:00|
 max|2019-12-31T23:59:00|
+-------+-------------------+

In [57]:
weather_df = weather.withColumn("DATE_IN_DATEFORMAT",weather['DATE'].cast(DateType()))
weather_df.select('DATE_IN_DATEFORMAT','DATE').show(10,False)

+------------------+-------------------+
DATE_IN_DATEFORMAT|DATE |
+------------------+-------------------+
2016-02-14 |2016-02-14T09:00:00|
2016-02-14 |2016-02-14T12:00:00|
2016-02-14 |2016-02-14T15:00:00|
2016-02-14 |2016-02-14T18:00:00|
2016-02-14 |2016-02-14T21:00:00|
2016-02-15 |2016-02-15T00:00:00|
2016-02-15 |2016-02-15T03:00:00|
2016-02-15 |2016-02-15T06:00:00|
2016-02-15 |2016-02-15T09:00:00|
2016-02-15 |2016-02-15T18:00:00|
+------------------+-------------------+
only showing top 10 rows

In [58]:
display(weather_df.sample(False, 0.0000001))

STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,WND,CIG,VIS,TMP,DEW,SLP,AA1,AA2,AJ1,AY1,AY2,GA1,GA2,GA3,GE1,GF1,IA1,KA1,KA2,MA1,MD1,MW1,OC1,OD1,SA1,UA1,REM,EQD,DATE_IN_DATEFORMAT
11302099999,2018-11-09T08:00:00,4,47.4333333,9.7333333,410.0,"DORNBIRN, AU",FM-12,99999,V020,"250,1,N,0010,1","99999,9,9,N","999999,9,9,9","+0089,1","+0074,1","10181,1",null,null,null,null,null,null,null,"99999,9,09695,1","3,1,007,1,+999,9",null,null,SYN05811302 35/// /2501 10089 20074 39695 40181 53007 333 55300=,null,null,null,null,null,null,null,null,null,null,2018-11-09
06669099999,2016-01-28T19:00:00,4,47.4833333,8.4,843.0,"LAEGEREN, SZ",FM-12,99999,V020,"999,9,C,0000,1","99999,9,9,N","999999,9,9,9","+0068,1","+0049,1","99999,9",null,"99999,9,09287,1","2,1,010,1,+999,9","3,1567,9","3,99,0057,1,999","5,99,0041,1,999",SYN07606669 43/// /23// 10068 20049 39287 48567 52010 333 55300 20007 91111 91208=,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2016-01-28
02535099999,2016-05-28T04:20:00,4,58.4564,13.972672,98.75,"SKOVDE, SW",FM-15,99999,V020,"999,9,V,0010,1","00183,1,9,N","009999,1,9,9","+0100,1","+0080,1","99999,9",null,null,"07,1,+00183,1,99,9","08,1,+00244,1,99,9",null,"9,AGL ,+99999,+99999","99,99,9,07,1,99,9,00183,1,99,9,99,9","10210,1,99999,9",null,null,MET069METAR ESGR 280420Z AUTO VRB02KT 9999 BKN006/// OVC008/// 10/08 Q1021=,null,null,null,null,null,null,null,null,null,null,null,2016-05-28
02807099999,2016-06-29T12:00:00,4,68.607269,27.405328,146.6,"IVALO, FI",FM-12,99999,V020,"220,1,N,0031,1","01340,1,9,9","045000,1,9,9","+0217,1","+0096,1","10135,1","00,1",null,null,null,"99,9,+01340,1,99,9",null,null,null,"9,AGL ,+99999,+99999","07,99,1,99,9,99,9,01340,1,99,9,99,9",null,null,"99999,9,09964,1",null,null,"0067,1",null,SYN004BUFR,null,null,null,null,2016-06-29
02835099999,2019-02-09T03:00:00,4,68.85,28.3,123.0,"INARI NELLIM, FI",FM-12,99999,V020,"290,1,N,0020,1","99999,9,9,N","007000,1,9,9","-0192,1","-0211,1","09967,1",null,null,null,"71,1","08,1,+01250,1,99,9",null,"9,MSL ,+99999,+99999","08,99,1,08,1,99,9,01250,1,99,9,99,9",null,null,null,"99999,9,09804,1","7,1,006,1,+999,9",null,null,null,SYN06602835 47657 82902 11192 21211 39804 49967 57006 771// 88/// 90300=,null,null,null,null,null,2019-02-09
01205099999,2019-04-03T10:00:00,4,62.3333333,5.2666666,37.17,"SVINOY LH, NO",FM-12,99999,V020,"050,1,N,0070,1","99999,9,9,N","999999,9,9,9","+0060,1","+0037,1","10082,1",null,null,"99999,9,10036,1","2,1,003,1,+999,9",null,null,null,null,SYN04801205 46/// /0507 10060 20037 30036 40082 52003=,null,null,null,null,null,null,null,null,null,null,null,null,null,2019-04-03
03862099999,2016-03-15T16:00:00,4,50.78,-1.8425,11.58,"BOURNEMOUTH, UK",FM-12,99999,V020,"050,1,N,0072,1","00690,1,9,N","017000,1,9,9","+0106,1","+0054,1","10280,1",null,null,null,null,null,null,"05,1,+00690,1,99,9",null,null,"9,AGL ,+99999,+99999","99,99,9,05,1,99,9,00690,1,99,9,99,9",null,null,"99999,9,10267,1","6,1,007,1,+999,9",null,null,null,null,null,SYN07003862 45567 /0514 10106 20054 30267 40280 56007 333 55305 21070 85/23=,null,2016-03-15
72302013748,2019-11-11T00:00:00,4,34.2675,-77.8997,10.1,"WILMINGTON INTERNATIONAL AIRPORT, NC US",FM-12,99999,V020,"999,9,C,0000,1","99999,9,9,N","016000,1,9,9","+0089,1","+0067,1","10213,1",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2019-11-11
60729099999,2017-12-27T21:00:00,4,36.4333333,10.0833333,156.0,"ZAGHONAN MAGRANE, TS",FM-12,99999,V020,"260,1,N,0051,1","99999,9,9,N","999999,9,9,9","+0121,1","+0076,1","10077,1","03,0000,9,1",null,null,null,null,null,null,null,null,null,null,null,null,null,"99999,9,09892,1","1,1,008,1,+999,9",null,null,"3,99,0103,1,999","4,99,0082,1,999",SYN07660729 26/// /2610 10121 20076 39892 40077 51008 333 59124 60007 91016 91120=,null,2017-12-27
02072099999,2016-12-13T05:00:00,4,67.7333333,22.8333333,313.0,"PARKALOMPOLO, SW",FM-12,99999,V020,"320,1,N,0010,1","02400,1,9,N","

In [59]:
# Join Airlines data and Weather data by DATE and AIRPORT

# Stations

In [61]:
stations = spark.read.option("header", "true").csv("dbfs:/mnt/mids-w261/data/DEMO8/gsod/stations.csv.gz")

In [62]:
display(stations)

usaf,wban,name,country,state,call,lat,lon,elev,begin,end
007018,99999,WXPOD 7018,null,null,null,0,0,+7018.0,20110309,20130730
007026,99999,WXPOD 7026,AF,null,null,0,0,+7026.0,20120713,20170822
007070,99999,WXPOD 7070,AF,null,null,0,0,+7070.0,20140923,20150926
008268,99999,WXPOD8278,AF,null,null,32.95,65.567,+1156.7,20100519,20120323
008307,99999,WXPOD 8318,AF,null,null,0,0,+8318.0,20100421,20100421
010016,99999,RORVIK/RYUM,NO,null,null,64.85,11.233,+0014.0,19870116,19910806
010017,99999,FRIGG,NO,null,ENFR,59.98,2.25,+0048.0,19880320,20050228
010071,99999,LONGYEARBYEN,SV,null,null,78.217,15.583,+0037.0,20050210,20050210
010190,99999,SVARTTANGEN,NO,null,null,77.517,20.817,+0020.0,20100825,20140523
010303,99999,TROMSO/SKATTURA,NO,null,null,69.7,19.017,+0014.0,20140522,20150108


In [63]:
from pyspark.sql import functions as f
stations.where(f.col('name').contains('JAN MAYEN NOR NAVY'))

Out[34]: DataFrame[usaf: string, wban: string, name: string, country: string, state: string, call: string, lat: string, lon: string, elev: string, begin: string, end: string]

In [64]:
stations.select('name').distinct().count()

Out[35]: 26497

In [65]:
display(stations.select('name').distinct())

name
HATTFJELLDAL-KRUTA
COLLAFIRTH HILL
WINDY HEAD
ST ATHAN
FINTHEN (USA-AF) &
HANAU AAF
WSCHOWA
WROCLAW/STRACHOWICE
VIGNA DI VALLE
SAMOS ISLAND


In [66]:
weather.select('NAME').distinct().count()

Out[37]: 14857

In [67]:
#display(weather.select('name').distinct())